In [1]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Carregar o dataset Iris
iris = load_iris()
X = iris.data
y = iris.target

# Dividir o dataset em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Padronizar as features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Treinar o modelo de regressão logística
model = LogisticRegression(multi_class='ovr', solver='lbfgs', max_iter=200)
model.fit(X_train, y_train)


LogisticRegression(max_iter=200, multi_class='ovr')

In [14]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from z3 import Optimize, Real, If, Sum, And, Bool, sat

# Carregar o dataset Iris
data = load_iris()
X = data.data
y = data.target

# Normalizar os dados
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Treinar o modelo de regressão logística
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200)
model.fit(X, y)

# Função para criar uma explicação mínima usando Z3 Solver
def minimal_explanation(model, instance, target_class, epsilon=1e-6):
    num_features = instance.shape[0]
    weights = model.coef_
    intercepts = model.intercept_
    
    # Z3 Optimize
    opt = Optimize()
    
    # Variáveis Z3
    feature_selection = [Bool(f'f{i}') for i in range(num_features)]
    
    # Função de decisão do modelo
    def decision_function(weights, intercept, instance, selected_features):
        return Sum([If(selected_features[i], instance[i] * weights[i], 0) for i in range(num_features)]) + intercept
    
    # Adicionar restrições ao solver
    for i in range(len(model.classes_)):
        if i != target_class:
            decision_target = decision_function(weights[target_class], intercepts[target_class], instance, feature_selection)
            decision_other = decision_function(weights[i], intercepts[i], instance, feature_selection)
            constraint = decision_target > decision_other + epsilon
            opt.add(constraint)
            print(f'Adding constraint for class {i}:')
            print(f'decision_target: {decision_target}')
            print(f'decision_other: {decision_other}')
            print(f'constraint: {constraint}')
    
    # Minimizar o número de características selecionadas
    opt.minimize(Sum([If(f, 1, 0) for f in feature_selection]))
    print("Objective: Minimize selected features")

    # Check satisfiability and get the model if possible
    result = opt.check()
    print(f"Solver result: {result}")
    
    if result == sat:
        m = opt.model()
        explanation = [i for i in range(num_features) if m.evaluate(feature_selection[i])]
        print("Model found. Explanation:", explanation)
        return explanation
    else:
        print("No solution found")
        return None

# Exemplo de uso
instance = X[0]  # Exemplo de instância para explicar
target_class = model.predict([instance])[0]  # Classe prevista pelo modelo

print("Instance:", instance)
print("Target class:", target_class)

explanation = minimal_explanation(model, instance, target_class)
print("Explicação mínima:", explanation)


Instance: [-0.90068117  1.01900435 -1.34022653 -1.3154443 ]
Target class: 0
Adding constraint for class 1:
decision_target: If(f0, 9696664878225023/10000000000000000, ToReal(0)) +
If(f1, 5909086383146621/5000000000000000, ToReal(0)) +
If(f2, 2584146221052813/1000000000000000, ToReal(0)) +
If(f3, 476788853622889/200000000000000, ToReal(0)) +
-639496772615327/3125000000000000
decision_other: If(f0, -531333707086969/1000000000000000, ToReal(0)) +
If(f1, -230954758158337/625000000000000, ToReal(0)) +
If(f2, 9773283627447047/20000000000000000, ToReal(0)) +
If(f3, 10878756066174153/10000000000000000, ToReal(0)) +
4149116688827359/2000000000000000
constraint: If(f0, 9696664878225023/10000000000000000, ToReal(0)) +
If(f1, 5909086383146621/5000000000000000, ToReal(0)) +
If(f2, 2584146221052813/1000000000000000, ToReal(0)) +
If(f3, 476788853622889/200000000000000, ToReal(0)) +
-639496772615327/3125000000000000 >
If(f0, -531333707086969/1000000000000000, ToReal(0)) +
If(f1, -230954758158337/62500

Explicação do Algoritmo
Inicialização do Solver: Inicializamos um solver Z3 e definimos variáveis booleanas para representar a seleção de cada característica.

Função de Decisão do Modelo: Definimos uma função de decisão que replica a decisão do modelo de regressão logística, mas apenas considera as características selecionadas.

Restrições de Classe: Adicionamos restrições ao solver para garantir que a função de decisão para a classe-alvo seja maior do que para todas as outras classes.

Minimização: Usamos o solver Z3 para minimizar o número de características selecionadas que satisfazem todas as restrições.

Interpretação do Modelo: Se o solver encontrar uma solução satisfatória, retornamos o subconjunto mínimo de características que explica a previsão do modelo.

Conclusão
Este é um exemplo básico de como usar o Z3 solver para encontrar explicações mínimas para um modelo de regressão logística em um problema de classificação multiclasse. Para um uso prático, ajustes adicionais e otimizações podem ser necessários, especialmente para lidar com conjuntos de dados maiores e mais complexos.

In [9]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from z3 import Optimize, Real, If, Sum, Bool, sat

# Carregar o dataset Iris
data = load_iris()
X = data.data
y = data.target

# Normalizar os dados
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Treinar o modelo de regressão logística
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200)
model.fit(X, y)

# Função para criar uma explicação mínima usando Z3 Solver
def minimal_explanation(model, instance, target_class, epsilon=1e-6):
    num_features = instance.shape[0]
    weights = model.coef_
    intercepts = model.intercept_
    
    # Z3 Optimize
    opt = Optimize()
    
    # Variáveis Z3
    feature_selection = [Bool(f'f{i}') for i in range(num_features)]
    
    # Função de decisão do modelo
    def decision_function(weights, intercept, instance, selected_features):
        return Sum([If(selected_features[i], instance[i] * weights[i], 0) for i in range(num_features)]) + intercept
    
    # Adicionar restrições ao solver
    for i in range(len(model.classes_)):
        if i != target_class:
            decision_target = decision_function(weights[target_class], intercepts[target_class], instance, feature_selection)
            decision_other = decision_function(weights[i], intercepts[i], instance, feature_selection)
            constraint = decision_target > decision_other + epsilon
            opt.add(constraint)
    
    # Minimizar o número de características selecionadas
    opt.minimize(Sum([If(f, 1, 0) for f in feature_selection]))

    # Check satisfiability and get the model if possible
    result = opt.check()
    
    if result == sat:
        m = opt.model()
        explanation = [i for i in range(num_features) if m.evaluate(feature_selection[i])]
        return result, explanation
    else:
        return result, None

# Exemplo de uso
instance = X[0]  # Exemplo de instância para explicar
target_class = model.predict([instance])[0]  # Classe prevista pelo modelo

result, explanation = minimal_explanation(model, instance, target_class)
if explanation:
    print("Objective: Minimize selected features")
    print("Solver result:", result)
    print("Model found. Explanation:", explanation)
    print("Explicação mínima:", explanation)
else:
    print("No solution found")


Objective: Minimize selected features
Solver result: sat
Model found. Explanation: [2, 3]
Explicação mínima: [2, 3]
